**Imports**

In [66]:
import tensorflow as tf
import seaborn as sf
import matplotlib as mtb
from keras.optimizers import RMSprop

from matplotlib import pyplot as pyp
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator


**Locations of the images**

The images are split in 2 directories, train and validation.
Each of those folders contain 2 directories, cats and dogs.

In [67]:
train_dir = "images/train"
validation_dir = "images/validation"

**Image generators**

Images will be turned into a 299, 299, 3 size.
Augmention will only occur in training, not validation.

In [68]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    batch_size=128,
    target_size=(299, 299, 3),
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    batch_size=20,
    target_size=(299, 299, 3),
    class_mode='binary'
)

Found 21002 images belonging to 2 classes.
Found 3998 images belonging to 2 classes.


**Base model**

The base model chosen here is the xeception model.
The input shape is set to (299, 299, 3).
The model's weights have been trained on imagenet dataset and will not be trained anymore.
The fully connected layers will be retrained though.

In [76]:
base_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(299, 299, 3),
    include_top=False)

**Freeze the model**

In [77]:
base_model.trainable = False

**Base model summary**

In [79]:
base_model.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_40 (InputLayer)           [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_40[0][0]                   
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

**Replacing fully connected layers**

The fully connected layers will be replaced with one with 1024 neuron layer followed by a 1 neuron layer for the output.

In [78]:
inputs = keras.Input(shape=(299, 299, 3))

flat1 = layers.Flatten()(base_model.outputs)
dense1 = layers.Dense(1024, activation='relu')(flat1)
dropout1 = layers.Dropout(0.2)(dense1)
output = layers.Dense(1, activation='sigmoid')

model = Model(inputs, output)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

model.summary()

AttributeError: 'list' object has no attribute 'shape'

**Full model summary**

In [ ]:
print(model)
model.summary()

**Fitting the model**


In [ ]:
history = model.fit_generator(train_generator,
                              validation_data=validation_generator,
                              steps_per_epoch=100,
                              epochs=20,
                              validation_steps=50,
                              verbose=2
)


In [73]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
# load model without classifier layers
model = VGG16(include_top=False, input_shape=(300, 300, 3))
print(model.outputs)
# add new classifier layers
flat1 = Flatten()(model.outputs)
dense1 = Dense(1024, activation='relu')(flat1)
output = Dense(10, activation='softmax')(dense1)
# define new model
model = Model(inputs=model.inputs, outputs=output)
# summarize
# model.summary()
# ...

[<tf.Tensor 'block5_pool_2/MaxPool:0' shape=(None, 9, 9, 512) dtype=float32>]
